In [10]:
from analysis_utils import *
%load_ext autoreload
import numpy as np
import pandas as pd

ptRNA = pd.read_excel('tRNAValues.xlsx',header=None)
gr04_ptRNA=ptRNA[0]
gr07_ptRNA=ptRNA[1]
gr107_ptRNA=ptRNA[2]
gr16_ptRNA=ptRNA[3]
gr25_ptRNA=ptRNA[4]

pCodon = pd.read_excel('codonValues.xlsx',header=None)
gr04_pCodon = pCodon[0]
gr07_pCodon = pCodon[1]
gr107_pCodon = pCodon[2]
gr16_pCodon = pCodon[3]
gr25_pCodon = pCodon[4]

rxndiff=dict()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
def computeBiasedElongationLatency(biasVal=1):
    sim_time_all = list()
    sim_time_std_all = list()
    rib_gr = [4,8,9,9,8,7]
    ptRNA = [gr07_ptRNA,gr107_ptRNA,gr16_ptRNA,gr16_ptRNA,gr25_ptRNA,gr25_ptRNA]
    pCodon =  [gr07_pCodon,gr107_pCodon,gr16_pCodon,gr16_pCodon,gr25_pCodon,gr25_pCodon]

    for i,num_rib in enumerate(rib_gr):
        weight = cognateDistrib(ptRNA[i],pCodon[i])
        sim_time_gr = list()
        sim_time_gr_var=list()
        rxn_count_gr = list()

        num_tRNA=42
        for num_tRNA in np.arange(1,42):
            sim_times=list()
            rxn_count = list()
            N=1000
            for i in range(N):
                rxn = eventbased_sim(rib_num=num_rib,tRNA_cog=num_tRNA,repeatAllowed=True,bias=biasVal)
                sim_times.append(rxn[0])
                rxn_count.append(rxn[1])
            sim_time_gr.append(np.average(sim_times))
            rxn_count_gr.append(np.average(rxn_count))
            sim_time_gr_var.append(np.std(sim_times)**2)

        #print(np.sum(sim_time_gr*weight[1:len(sim_time_gr)+1]))
        sim_time_all.append(np.sum(sim_time_gr*weight[1:len(sim_time_gr)+1]))
        sim_time_std_all.append(np.sqrt(np.sum((sim_time_gr_var)*weight[1:len(sim_time_gr)+1]))/np.sqrt(num_tRNA*N))
    return(sim_time_all, sim_time_std_all)


In [20]:
elongLatencyList = list()
elongLatency_STDlist = list()
bias = np.arange(1,10,0.5)

for biasVal in bias:
    elongLatency, elongLatency_STD = computeBiasedElongationLatency(biasVal)
    elongLatencyList.append(elongLatency)
    elongLatency_STDlist.append(elongLatency_STD)
    print("\n\nBias: ", biasVal, "\n", elongLatency, elongLatency_STD)



Bias:  1.0 
 [37.63759268889624, 38.320504226913734, 36.98292889637495, 37.44690694617251, 35.67909294797215, 36.246404486001616] [0.2555239736453694, 0.2511613758593155, 0.25174408096348694, 0.24588885030176924, 0.23314029750878662, 0.2487483465026793]


Bias:  1.5 
 [24.36417456610702, 24.276044204058948, 22.843728802151887, 24.133837091388116, 23.753116633877294, 23.793360290491055] [0.16442824994385244, 0.15606078286048414, 0.1533695447851116, 0.15975970166420855, 0.1550998337516166, 0.15732410120635335]


Bias:  2.0 
 [18.55239448485358, 17.496836493441414, 16.84370299474137, 16.759298691363888, 17.70858002323581, 17.710808682632013] [0.12936723917746792, 0.11534063087534994, 0.10993900113543817, 0.1109532926988609, 0.11801505838157665, 0.11862782576583489]


Bias:  2.5 
 [14.726554552292944, 13.755676963412451, 13.723748117819614, 13.718034049079872, 13.622079591530762, 13.808922636925468] [0.0959399489769176, 0.08892800620076124, 0.09080324711335722, 0.08791886105557849, 0.086